# This notebook provides portfolio weights based on the historical data to the backtesting notebook

In [1]:
# !pip uninstall deepdowmine
#!pip install git+https://github.com/dsman1823/deepdowmine.git
#!conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch # <---- to Enable SVD 

In [18]:
import torch

import pandas as pd
import numpy as np

from datetime import datetime 
from deepdowmine.nn import LinearNetMine, UpdNumericalMarkowitzWithShorting, UpdLinearNetMine, UpdDenseNet, UpdUpdLinearNetMine
from deepdowmine.nn import MinVarDenseNet, ConvNetFullOpti, ConvNetMinVar, RnnNetMinVar, LstmNetMinVar, RnnNetFullOpti
from deepdowmine.nn import LstmNetFullOpti

In [2]:
RETS_FILE_PATH = 'historical_returns.csv'



loockback, gap, horizon = 50, 0, 5
n_assets = 5
loockback = 50


## Load NN from dict

In [3]:
LINEAR_NET_OSHARPE_FILE_PATH = r'./NNs/linear_net_50x5_sharpe.pth' # LinearNetMine with ordinary Sharpe cost

In [4]:
# years parameter reprsenets amount of years after the backtest stasrt: the model dowloaded is the the model trained 
# on the period (train_start, back_test_start + years) 


# LinearNetMine with ordinary sharpe cost function
def load_linear_net_osharpe(years=0):
    network = LinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{years}.pth'))
    print(fr'./NNs/linear_net_50x5_sharpe_{years}.pth')
    return network.eval()
    

In [5]:
def load_dense_net(years=0):
    network = UpdLinearNetMine(1, loockback, n_assets, p=0.5)
    network.load_state_dict(torch.load(fr'./NNs/dense_{years}.pth'))
    print(fr'dense_{years}.pth')
    return network.eval()

## Core functions

In [6]:
def transform_rets_to_NN_input(rets):
    # tranform (loockback, n_assets) df into (1, 1, loockback, n_assets)=(sample_size, n_channels, loockback, n_assets) tensor 
    returns_np = rets.to_numpy()

    # Add the required dimensions: (n_samples, n_channels, lookback, n_assets)
    returns_np_expanded = np.expand_dims(returns_np, axis=0)  # Adds n_samples dimension
    returns_np_expanded = np.expand_dims(returns_np_expanded, axis=0)  
    return torch.from_numpy(returns_np_expanded).float()

def weights_from_NN(rets, network):
    X = transform_rets_to_NN_input(rets)
    return network(X).detach().numpy()[0]

# method is a function, used for obtaining the returns
def get_weights(method):
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    return method(rets)

In [7]:
rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
X = transform_rets_to_NN_input(rets)
network = LinearNetMine(1, loockback, n_assets, p=0.5)
network.load_state_dict(torch.load(fr'./NNs/linear_net_50x5_sharpe_{2}.pth'))
network.eval()(X).detach().numpy()[0]

array([-0.0160506 ,  0.20566106,  0.03464727,  0.3581981 ,  0.41754425],
      dtype=float32)

## NNs and related methods


### NNs

networks[i] method is trained on (train_start, backtest_start + i years) data interval

In [8]:
years = [0, 1, 2]#, 1, 2]

linear_osharpe_networks = [load_linear_net_osharpe(years=i) for i in years]
dense_networks = [load_dense_net(years=i) for i in years]

./NNs/linear_net_50x5_sharpe_0.pth
./NNs/linear_net_50x5_sharpe_1.pth
./NNs/linear_net_50x5_sharpe_2.pth
dense_0.pth
dense_1.pth
dense_2.pth


### methods

In [9]:
dense_networks[1].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [10]:
dense_networks[2].linear0.bias[0:3]

tensor([ 0.0484,  0.0029, -0.0442], grad_fn=<SliceBackward0>)

In [11]:
# note the format of lambda 
linear_osharpe_methods = [lambda r, i=i: weights_from_NN(r, linear_osharpe_networks[i]) for i in years] 
dense_methods = [lambda r, i=i: weights_from_NN(r, dense_networks[i]) for i in years] 


In [12]:
[get_weights(dense_methods[i]) for i in years]

[array([-0.11055882,  1.0000925 ,  0.06984006,  0.02121476,  0.01941148],
       dtype=float32),
 array([-0.19172463,  1.        ,  0.0724505 ,  0.01942461,  0.09984952],
       dtype=float32),
 array([-0.1401442 ,  0.9999889 ,  0.0180865 , -0.02224984,  0.1443186 ],
       dtype=float32)]

# Server

In [13]:
from dateutil.relativedelta import relativedelta
from datetime import datetime, date

RETRAINING_DATE1 = date(2021, 12, 15)
RETRAINING_DATE2 = date(2022, 12, 15)

In [14]:
tmp = True
not tmp

False

In [29]:
from flask import Flask, request, jsonify
from flask_cors import CORS 

import numpy as np

app = Flask(__name__)
CORS(app)  # Enable CORS for your Flask app

markers = {}
markers['net_initialized'] = False

@app.route('/get_weights', methods=['GET'])
def get_request():
    # Extract date string from the query parameters
    date_str = request.args.get('date', None)
    
    # Simple validation to check if date is provided
    if not date_str:
        return jsonify({"error": "Missing date parameter"}), 400

    # Try to convert the date string to a datetime object
    try:
        # Note that now we're only parsing the date, not the time
        date = datetime.strptime(date_str, "%Y-%m-%d").date()
    except ValueError:
        # If there is an error in parsing the date, return an error message
        return jsonify({"error": "Invalid date format. Please use YYYY-MM-DD format."}), 400
    
#     years_plus = 0
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         years_plus = 1
#         print(f'Working with model: {years_plus}')
#     if RETRAINING_DATE2 < date:
#         years_plus = 2
#         print(f'Working with model: {years_plus}')
    
#     weights = get_weights(linear_osharpe_methods[years_plus]).tolist()
    
    rets = pd.read_csv(RETS_FILE_PATH, index_col=0)
    X = transform_rets_to_NN_input(rets)
    
    network = LstmNetMinVar(5)
    network.load_state_dict(torch.load(fr'./NNs/trained_on_new_data/lstm_minvar.pth'))
       
#     if date < RETRAINING_DATE1:
#         print(0)
#     if RETRAINING_DATE1 < date < RETRAINING_DATE2:
#         print('1')
#         network = MinVarDenseNet(1, 50, 5)
#         network.load_state_dict(torch.load(fr'./NNs/min_var_dense_net_1.pth'))
#     if RETRAINING_DATE2 < date:
#         network = MinVarDenseNet(1, 50, 5)
#         network.load_state_dict(torch.load(fr'./NNs/min_var_dense_net_2.pth'))
 #       print('2')
    
    weights = network.eval()(X).detach().numpy()[0].tolist()
    
        
    #weights = weights_from_NN(rets, linear_osharpe_networks[0]).tolist()
    #print(weights)
    # Return the vector as JSON, using the string representation of the date for simplicity
    return jsonify({'weights': weights})


In [ ]:
app.run(port=5000)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
127.0.0.1 - - [17/Mar/2024 20:50:51] "GET /get_weights?date=2017-12-18 HTTP/1.1" 200 -
C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
127.0.0.1 - - [17/Mar/2024 20:50:53] "GET /get_weights?date=2017-12-26 HTTP/1.1" 200 -
C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all bu

C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
127.0.0.1 - - [17/Mar/2024 20:51:34] "GET /get_weights?date=2018-05-07 HTTP/1.1" 200 -
C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "
127.0.0.1 - - [17/Mar/2024 20:51:36] "GET /get_weights?date=2018-05-14 HTTP/1.1" 200 -
C:\Users\seidm\miniconda3\envs\ddow\lib\site-packages\torch\nn\modules\rnn.py:71: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_l

In [ ]:
update the 